
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [1]:
# python imports
import re
import json
from os.path import join

# third party imports
import pandas
import escher
import cobra.test

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import ribosome, tRNA_charging, transcription

from minime.util import dogma
from minime import *
from minime.util import building
from minime.util.building import add_complex_stoichiometry_data, add_transcription_reaction
from minime.util.building import add_complex_modification_data
from minime.util.building import add_metabolic_reactions
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

In [2]:
import ecolime

In [3]:
ijo = cobra.test.create_test_model('ecoli')

# Colton TODOs 
- [x] 1) look into all of the free modifications in the original ME and make sure none of thes reactions are blocked
- [x] 2) Frameshifts (b2891)
- [x] 3) selenocysteine
- [x] 4) Non AUG start codons (15%)
- [x] 5) moaD is a modifier protein that never has to be made
- [x] 6) Correct rpL7/12_mod_1:acetyl modification
    - in iOL as a free modification. could be corrected
- 7) Remaining Genes: 
     - pflC, citC, norW - formed but not used in iJL1678
     - FusA, tsf - translation reactions need audited and improved
     - paaD - FeS transfer chaperone. This process needs audited and improved
     - pnp, orn, rhlB - degradosome
     - secB, tatE - Translocation Pathways
     - lgt, lspA - part of translocation. Not sure how to incorporate-ask Joanne about this

In [4]:
from minime.util.mass import compute_RNA_mass

## Begin by loading metabolites and build Metabolic reactions

In [5]:
me = MEmodel('iJO1366-ME')
m_model = ecolime.get_m_model()
# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complex_list = []
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("CPLX"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("EG"))
complex_list.extend(i.id for i in m_model.metabolites if "-MONOMER" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "-CPLX" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "_mod_" in i.id)
# temp fix
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Isc"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Suf"))
complex_list = set(complex_list)
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_list)

# if the bounds of this metabolite aren't open, model uses wrong reactions
me.reactions.EX_pqq_e.lower_bound = -1000


Assigned tl to c
Assigned hemed to c
Assigned pqq to c
Assigned dpm to c
Assigned 23bpg to c
Assigned tqn to c


In [6]:
me.translation_info["translation_terminators"] = ribosome.translation_stop_dict
me.translation_info["met_start_codons"] = {"AUG", "GUG", "UUG", "AUU", "CUG"}

In [7]:
essential_list = [
    'trdox_c',
    'fldox_c',
    'LI_c',
    'cs_e',
    'fldrd_c',
    'trdrd_c',
    'Oxidized_c',
    'btn_c',
    '3fe4s_c',
    'palmitate_c',
    'C10H8O5_c', 'C9H9O4_c', # for tRNA modifications
    'NiFeCoCN2_c', 'acetyl_c',
    'RNase_m5','RNase_m16','RNase_m23'] # RNAses are gaps in model, fix how cs is added to

for met_id in essential_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

unknown metabolite 'trdox_c' created
unknown metabolite 'fldox_c' created
unknown metabolite 'LI_c' created
unknown metabolite 'fldrd_c' created
unknown metabolite 'trdrd_c' created
unknown metabolite 'Oxidized_c' created
unknown metabolite '3fe4s_c' created
unknown metabolite 'palmitate_c' created
unknown metabolite 'C10H8O5_c' created
unknown metabolite 'C9H9O4_c' created
unknown metabolite 'NiFeCoCN2_c' created
unknown metabolite 'acetyl_c' created
unknown metabolite 'RNase_m5' created
unknown metabolite 'RNase_m16' created
unknown metabolite 'RNase_m23' created


In [8]:
tRNA_mod_dict = get_tRNA_modifications()

In [9]:
for mod, components in ecolime.tRNA_charging.trna_modification.iteritems():
    tRNA_mod = ModificationData(mod, me)
    tRNA_mod.enzyme = components['machines']
    tRNA_mod.stoichiometry = components['metabolites']
    if 'carriers' in components.keys():
        for carrier, stoich in components['carriers'].items():
            if stoich < 0:
                tRNA_mod.enzyme += [carrier]
            tRNA_mod.stoichiometry[carrier] = stoich

## Build ribosome and RNA Polymerase

### Add translation-related Subreactions

In [10]:
for subreaction in ribosome.translation_subreactions:
    subreaction_data = SubreactionData(subreaction, me)
    subreaction_data.enzyme = ribosome.translation_subreactions[subreaction]['enzyme']

for subreaction in transcription.transcription_subreactions:
    subreaction_data = SubreactionData(subreaction, me)
    subreaction_data.enzyme = transcription.transcription_subreactions[subreaction]['enzymes']
    subreaction_data.stoichiometry = transcription.transcription_subreactions[subreaction]['stoich']

Selenocysteine and start codons

In [11]:
# TODO contrant type is 'trna efficiency' does this change anything?
# 1 machine + 1 atp + 1 aa + 1 h2o --> 1 machine-amp + 1 h + 1 ppi
# 1 machine-amp + 1 free tRNA --> 1 machine + 1 amp + 1 charged tRNA
subreaction_data = SubreactionData('fmet_addition_at_START', me)
subreaction_data.enzyme = ['InfB_mono',  # Start codon loader enzyme and formylmethyltransferase
                           'Fmt_mono_mod_mg2_mod_k'] 
subreaction_data.stoichiometry = {'h_c': 1, '10fthf_c': -1, 'thf_c':1,
                                 'generic_tRNA_START_met__L_c': -1, 'met__L_c': -1}

subreaction_data = SubreactionData('sec_addition_at_UGA', me)
subreaction_data.enzyme = ['SelA_deca_mod_10:pydx5p',
                           'SelB_mono'] # Selenocysteine loader enzyme
subreaction_data.stoichiometry = {'h_c': 1, 'h2o_c': -1, 'selnp_c':-1, 'pi_c':1,
                                 'generic_tRNA_UGA_cys__L_c': -1, 'cys__L_c': -1}

# create subreaction for each codon
for codon in dogma.codon_table:
    if dogma.codon_table[codon] == '*':
        stop_codon = codon.replace('T','U')
        stop_enzyme = ribosome.translation_stop_dict.get(stop_codon)
        me.add_metabolites([Complex(stop_enzyme)])
        
        subreaction_data = SubreactionData(stop_codon + '_' + stop_enzyme +
                                           '_mediated_termination', me)
        subreaction_data.enzyme = stop_enzyme
        subreaction_data.stoichiometry = {}
    else:
        full_aa = dogma.amino_acids[dogma.codon_table[codon]]
        amino_acid = full_aa.split('_')[0]
        subreaction_data = SubreactionData(amino_acid + '_addition_at_' \
                                           + codon.replace('T','U'), me)
        tRNA = 'generic_tRNA_' + codon.replace('T','U') + '_' + full_aa
        subreaction_data.enzyme = 'generic_Tuf' # Default AA loader enzyme
        subreaction_data.stoichiometry = {full_aa: -1, 'atp_c': -1,# 'h2o_c': 1,
                                          'amp_c': 1, 'h_c': 1, 'ppi_c': 1,
                                          tRNA: -1}

In [12]:
gb_filename = join(ecoli_files_dir,'NC_000913.2.gb')                                                                                    
TU_df = pandas.read_csv(join(ecoli_files_dir,'TUs_from_ecocyc.txt'), delimiter="\t", index_col=0)


sigma_to_RNAP_dict = transcription.sigma_factor_complex_to_rna_polymerase_dict
building.build_reactions_from_genbank(me, gb_filename, TU_df, tRNA_modifications=tRNA_mod_dict, verbose=False,
                                      translation_terminators=ribosome.translation_stop_dict,
                                      frameshift_dict=ribosome.frameshift_dict,
                                     methionine_cleaved=ribosome.methionine_cleaved,
                                     folding_dict=ribosome.folding_dict,
                                     sigma_to_RNAP_dict=sigma_to_RNAP_dict,
                                     tRNA_to_codon=ribosome.tRNA_to_codon)

Applying frameshift on b2891
Created <TranscribedGene Rho_hexa_mod_3:mg2 at 0x7fd3e659abd0> in <TranscriptionReaction transcription_TU0_13350_from_RpoD_mono at 0x7fd3ea5e25d0>

/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:265 UserWarning: RNA Polymerase (RNAP70-CPLX) not found
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:265 UserWarning: RNA Polymerase (CPLX0-222) not found
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:265 UserWarning: RNA Polymerase (RNAP54-CPLX) not found
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:265 UserWarning: RNA Polymerase (RNAPE-CPLX) not found
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:265 UserWarning: RNA Polymerase (RNAP32-CPLX) not found
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:265 UserWarning: RNA Polymerase (RNAPS-CPLX) not found
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:265 UserWarning: RNA Polymerase (CPLX0-221) not found



Created <TranscribedGene Mfd_mono_mod_1:mg2 at 0x7fd3e64d2310> in <TranscriptionReaction transcription_TU0_13350_from_RpoD_mono at 0x7fd3ea5e25d0>
Created <TranscribedGene RpoZ_mono_mod_1:mg2 at 0x7fd3e64d2390> in <TranscriptionReaction transcription_TU0_13350_from_RpoD_mono at 0x7fd3ea5e25d0>
Created <TranscribedGene NusG_mono at 0x7fd3e64d2350> in <TranscriptionReaction transcription_TU0_13350_from_RpoD_mono at 0x7fd3ea5e25d0>
Created <TranscribedGene GreA_mono at 0x7fd3e64d23d0> in <TranscriptionReaction transcription_TU0_13350_from_RpoD_mono at 0x7fd3ea5e25d0>
Created <TranscribedGene NusA_mono at 0x7fd3e64d2410> in <TranscriptionReaction transcription_TU0_13350_from_RpoD_mono at 0x7fd3ea5e25d0>
Created <TranscribedGene GreB_mono at 0x7fd3e64d2450> in <TranscriptionReaction transcription_TU0_13350_from_RpoD_mono at 0x7fd3ea5e25d0>
Created <TranscribedGene RpsJ_mono at 0x7fd3e64d2490> in <TranscriptionReaction transcription_TU0_14574_with_TERM0_1066_from_RpoD_mono at 0x7fd3ea2edb10

/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:343 UserWarning: ribosome not found


In [13]:
ribosome.add_ribosome(me)
transcription.add_RNA_polymerase_complexes(me)

Created <Complex RlmI_dim at 0x7fd3e8cc0c50> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex RplA_mono at 0x7fd3e8cc0cd0> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex RsmD_mono at 0x7fd3e8cc0d90> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex RplN_mono at 0x7fd3e8cc0d50> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex RpsF_mono at 0x7fd3e8cc0d10> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex RrmJ_mono at 0x7fd3e8cc0dd0> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex YjbC_mono at 0x7fd3e8cc0e10> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex RrmA_dim_mod_2:zn2 at 0x7fd3e8cc0e50> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex Tig_mono at 0x7fd3e8cc0ed0> in <ComplexFormation formation_ribosome at 0x7fd3e8cc0c10>
Created <Complex RpsP_mono at 0x7fd3e8cc0e90> in

Make a dummy reactions. Add a dummy protein in as well.

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [14]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [15]:
# ME_complex_dict is a dict of {'complex_id': [{'bnum' : count}]}
rna_components = {"b3123"} # component id should have RNA_ instead of protein_
ME_complex_dict = get_complex_to_bnum_dict(rna_components)
add_complex_stoichiometry_data(me, ME_complex_dict)

In [16]:
modification_dict = get_protein_modification_dict()
add_complex_modification_data(me, modification_dict)

In [17]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
#alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.stoichiometry = cplx_data.stoichiometry
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications.copy()
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [18]:
# chaperones
# todo bmocogdp_c mods
# todo FE-S modifications
fes_chaperones = {'CPLX0-1762':'G6712-MONOMER'} # FE-S modification
                 
bmocogdp_chaperones = {'TMAOREDUCTI-CPLX':'EG12195-MONOMER', #bmocogdp
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER', #bmocogdp
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', #bmocogdp
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER', #bmocogdp
                 'NAP-CPLX':'NAPD-MONOMER', #bmocogdp
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'} #bmocogdp
for chaperone in set(bmocogdp_chaperones.values()):
    new_mod = ModificationData('mod_bmocogdp_c_' + chaperone, me)
    new_mod.enzyme = chaperone
    new_mod.stoichiometry = {'bmocogdp_c': -1}

for cplx_data in me.modification_data.get_by_id('mod_bmocogdp_c').get_complex_data():
    cplx_id = cplx_data.id.split('_mod')[0]
    if cplx_id in bmocogdp_chaperones:
        cplx_data.modifications['mod_bmocogdp_c_' + bmocogdp_chaperones[cplx_id]] = \
            cplx_data.modifications.pop('mod_bmocogdp_c')

## TODO NEXT complete Iron-Sulfur cluster modification reactions. Apply rRNA/tRNA fraction constraint 

 - Read "The CO and CN− ligands to the active site Fe in [NiFe]-hydrogenase of Escherichia coli have different metabolic origins" for insight into the metabolic origin of the NiFeCoCN2_c group

Build all complex formation reactions

In [19]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

In [20]:
building.add_dummy_reactions(me, "ATG" + "TTT" * 12 + "TAT" * 12 + "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TGA", update=False)

TODO:
 - filter so known erpA dependent ones use that instead

In [21]:
# these guys can transfer assembled iron sulfur clusters to the various enzymes
fes_transfer = {"erpA": "CPLX0-7617", "iscA": "IscA_tetra", "sufA": "CPLX0-7824"}

suf_cplx = ComplexData("sufBC2DES_pathway_complex", me)
suf_cplx.stoichiometry = {"CPLX0-1341": 1, "CPLX0-246_CPLX0-1342_mod_pydx5p": 1}
suf_cplx.create_complex_formation()
    
isc_cplx = ComplexData("iscUS_cyaY_pathway_complex", me) # could add chaperones into here
isc_cplx.stoichiometry = {"IscU": 1, "IscS_mod_2:pydx5p": 1, "EG11653-MONOMER": 1}
isc_cplx.create_complex_formation()

generic_fes_transfer = GenericData("generic_fes_transfer", me, ['CPLX0-7617', 'CPLX0-7824', 'IscA_tetra'])
generic_fes_transfer.create_reactions()

#building.add_and_update_metabolic_reaction(me, "isc_2fe2s_formation", suf_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "suf_2fe2s_formation", isc_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "isc_4fe4s_formation", suf_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "suf_4fe4s_formation", isc_cplx.id, False, update=True)

In [22]:
me.modification_data.mod_2fe2s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_2fe2s_c.keff = 65.
me.modification_data.mod_4fe4s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_4fe4s_c.keff = 65.

Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [23]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_complex(generic=False)

Fixed _DASH: D__LACtex
Fixed _DASH: L__LACD2
Fixed _DASH: L__LACD3
Fixed _DASH: D__LACt2pp
Fixed _DASH: L__LACtex
Fixed _DASH: L__LACt2rpp
Assigned tl to c
Assigned hemed to c
Assigned pqq to c
Assigned dpm to c
Assigned 23bpg to c
Assigned tqn to c


In [24]:
rxn_info = get_reaction_info_frame()
for reaction_data in me.stoichiometric_data:
    add_metabolic_reactions(me, reaction_data, rxnToModCplxDict, rxn_info, update=True,
                            create_new=True)

Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

In [25]:
for generic, components in ecoli_k12.generic_dict.items():
    GenericData(generic, me, components).create_reactions()

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [26]:
apoACP = me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p')
for reaction in me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p').reactions:
    if apoACP in reaction.reactants:
        reaction.complex_dilution_list = {apoACP.id}
        reaction.update()
me.reactions.acp_lipoate_synthase_FWD_None.complex_dilution_list = {'CPLX0-782_mod_2:4fe4s'}
me.reactions.MOADSUx1_FWD_CPLX_dummy.complex_dilution_list = {'EG11597-MONOMER_mod_amp'}

## Add in translocation

In [27]:
translocPath = pandas.read_csv(join(ecoli_files_dir, "translocation_pathways.txt"),sep='\t')
for index, row in translocPath.iterrows():
    translocRxn = ProteinTranslocationData(row.Reaction_name, me)
    translocRxn.keff = float(row.Keff)
    translocRxn.costs_complexes = row.Complexes.split(' AND ')

In [28]:
proteins_sa_coeff_inner={}

transloc = pandas.read_csv(join(ecoli_files_dir, "peptide_compartment_and_pathways2.txt"), sep='\t', comment="#")
for index, row in transloc.iterrows():
    me.metabolites.get_by_id(row.Complex).compartment = row.Complex_compartment
    me.metabolites.get_by_id('protein_'+row.Protein.split('(')[0]).compartment = row.Protein_compartment
    pathway = False
    if 's' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        
        ## This is in preparation for membrane constraint
        mass = me.translation_data.get_by_id(row.Protein.split('(')[0]).mass
        if row.Complex in proteins_sa_coeff_inner.keys():
            proteins_sa_coeff_inner[row.Complex]+=mass*1.21/42.*2
        else:
            proteins_sa_coeff_inner[row.Complex]=mass*1.21/42.*2
    if 'r' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)       
    if 'p' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.srp_yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)         
    if 't' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.tat_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    if 'a' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        me.translocation_pathways.secA_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    if 'l' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.lol_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    if 'b' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.bam_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    if 'y' in row.translocase_pathway:
        pathway = True
        me.translocation_pathways.yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)        
    if not pathway:
        print row.Complex, row.translocase_pathway
    #if row.translocase_pathway != 'n':
    #    print row.Complex, row.translocase_pathway        

## Make RNA splicing machinery

In [29]:
excision_types = ['rRNA_containing', 'monocistronic',
                  'polycistronic_wout_rRNA']
for excision_type in excision_types:
    complex_data =  ComplexData(excision_type + "_excision_machinery", me)
    complex_data.stoichiometry = {i: 1 for i in ecoli_k12.excision_machinery[excision_type]}
    complex_data.create_complex_formation()
    modification = ModificationData(excision_type + "_excision", me)
    modification.stoichiometry = {'h2o_c': -1, 'h_c': 1}
    modification.enzyme = complex_data.id

for t in me.transcription_data:
    n_excised = sum(t.excised_bases.values())
    n_cuts = len(t.RNA_products) * 2
    if n_excised == 0 or n_cuts == 0:
        continue
    RNA_types = list(t.RNA_types)
    n_tRNA = RNA_types.count("tRNA")
    
    if "rRNA" in set(RNA_types):
        t.modifications["rRNA_containing_excision"] = n_cuts
    elif n_tRNA == 1:
        t.modifications["monocistronic_excision"] = n_cuts
    elif n_tRNA > 1:
        t.modifications["polycistronic_wout_rRNA_excision"] = n_cuts
    else:  # only applies to rnpB
        t.modifications["monocistronic_excision"] = n_cuts

## Attempt to set keffs

In [30]:
keffs = get_reaction_keffs(me, verbose=True)
for reaction_id, keff in keffs.items():
    me.reactions.get_by_id(reaction_id).keff = keff

no keff found for keff_PGK_PGK_DASH_CPLX
no keff found for keff_PGK_PGK_DASH_CPLX
no keff found for keff_TPI_TPI_DASH_CPLX
no keff found for keff_TPI_TPI_DASH_CPLX
no keff found for keff_GLUTRR_CPLX0_DASH_3741
no keff found for keff_suf_2fe2s_formation_sufBC2DES_pathway_complex
no keff found for keff_suf_4fe4s_formation_sufBC2DES_pathway_complex
no keff found for keff_isc_2fe2s_formation_iscUS_cyaY_pathway_complex
no keff found for keff_isc_4fe4s_formation_iscUS_cyaY_pathway_complex


Figure out why these are necessary

In [31]:
r = Reaction('DM_preq1_c')
me.add_reaction(r)
r.reaction = 'preq1_c' + ' <-'

r = Reaction('DM_ahcys_c')
me.add_reaction(r)
r.reaction = 'ahcys_c' + ' ->'

r = Reaction('DM_amet_c')
me.add_reaction(r)
r.reaction = 'amet_c' + ' <-'

r = Reaction('DM_dmpp_c')
me.add_reaction(r)
r.reaction = 'dmpp_c' + ' <-'

# TODO address these 

make sure they are actually dead ends

In [32]:
me.update()
me.prune()

Adding selenocystein for RNA_b1474
Adding selenocystein for RNA_b3894
Adding selenocystein for RNA_b4079


In [33]:
for metabolites in me.metabolites:
    if len(metabolites.reactions) > 1:
        continue
    try:
        if len(metabolites.reactions) == 1 and len(ijo.metabolites.get_by_id(metabolites.id).reactions) > 1:
            print metabolites.id, metabolites.reactions

    except:
        print metabolites.id, [i.id for i in metabolites.reactions][0]

mococdp_c frozenset([<MetabolicReaction MOCDS_FWD_G7496-MONOMER_mod_mg2 at 0x7fd3e7f4d910>])
tungs_c frozenset([<MetabolicReaction TUNGSabcpp_FWD_ABC-19-CPLX at 0x7fd3e8118c90>])


## Clean up and update everything

Figure out why these are necessary

glycogen_demand = (glycogen_g_per_g)*(1./glycogen_mw)*1000. 

In [34]:
n_genes = len(me.metabolites.query('RNA_b'))
print("number of genes in the model %d (%.2f%%)" % (n_genes, n_genes * 100. / (1678)))

number of genes in the model 1665 (99.23%)


One of these demand metabolites are blocked....

In [35]:
demands = {'murein5px4p_p': 0.01389, 'kdo2lipid4_e': 0.01945, 'pe160_c': 0.01786,
           'pe160_p': 0.04594, 'pe161_c': 0.02105, 'pe161_c': 0.05415}
for met, stoich in demands.items():
    r = Reaction('DM_' + met)
    r.add_metabolites({me.metabolites.get_by_id(met): -1})
    r.lower_bound = stoich
    me.add_reaction(r)

## Solve

In [36]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [37]:
# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMp']
for reaction in KO_list:
    a = me.reactions.query(reaction + '_')
    for rxn in a:
        rxn.upper_bound = 0
        rxn.lower_bound = 0

Save for future recomputation

In [38]:
expressions = compile_expressions(me)

In [39]:
solve_at_growth_rate(me, .1, compiled_expressions=expressions)

<Solution 0.10 at 0x7fd3e24fc950>

In [40]:
binary_search(me, compiled_expressions=expressions, debug=False)

mu		status
0.0000000000	+
2.0000000000	-
1.0000000000	-
0.5000000000	+
0.7500000000	+
0.8750000000	+
0.9375000000	-
0.9062500000	+
0.9218750000	-
0.9140625000	-
0.9101562500	-
0.9082031250	+
0.9091796875	-
0.9086914062	+
0.9089355469	-
0.9088134766	-
0.9087524414	-
0.9087219238	-
0.9087066650	-
0.9086990356	-
0.9086952209	+
0.9086971283	+
0.9086980820	-
0.9086976051	-
0.9086973667	+
0.9086974859	-
0.9086974263	+
0.9086974561	-
0.9086974412	+
0.9086974487	-
0.9086974449	-
0.9086974431	-
0.9086974422	-
0.9086974412	+
completed in 312.2 seconds and 34 iterations


<Solution 0.91 at 0x7fd3e24fce10>